In [2]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/anura/anaconda3/Library/bin/graphviz'
import sys
import networkx as nx
import numpy as np
import random
import pickle
import sys
from numba import jit
import elfi
import sklearn as sk
import scipy
from scipy.spatial import distance
import pandas as pd
import sys
print(sys.version)

3.7.9 (default, Aug 31 2020, 12:42:55) 
[GCC 7.3.0]


In [3]:
#for i in range(len(sglist)):
#    print('# of nodes in {i}th component is  - ', str(np.mean(gmat[i])))
seeds = pd.read_csv("../data/icpsr/DS0001/paluck-seed.csv")
seeds['ID'] = ((seeds['SCHIDW2'] * 1000) + pd.to_numeric(seeds['ID'], errors='coerce'))

In [4]:
es = pd.read_csv("../data/icpsr/DS0001/paluck-endstate.csv")
es['ID'] = ((es['SCHIDW2'] * 1000) + pd.to_numeric(es['ID'], errors='coerce'))

 /home/gary/.conda/envs/elfi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning:Columns (238,242,244,245,246,247,248,249,382) have mixed types.Specify dtype option on import or set low_memory=False.


In [5]:
def genNet(n, k=4, pRewire=.1, type='grid'):
    # create net
    if type == 'grid': #wrap the grid
        net = nx.grid_2d_graph(int(n**.5), int(n**.5), periodic=True)
        #net = nx.grid_2d_graph(3, 3, periodic=True)
        #net = nx.grid_2d_graph(2, 2, periodic=False)
        #plotNet(net)
        #print(len(net.edges()))
        #assert(0)
        # rewire
        numRewired = 0
        #while numRewired < (pRewire * nx.number_of_nodes(net)):
        while numRewired < 1:
            tries = 0
            while tries < 100:
                tries = tries + 1
                #print([numRewired, pRewire * nx.number_of_nodes(net), tries])
                v1 = random.choice(net.nodes())
                v2 = random.choice(net.nodes())
                if not( net.has_edge(v1,v2) or v1==v2 or len(net.neighbors(v1)) <= 1): #net.neighbors is sometimes (often?) a blank set, changed so v1 needs 2 nb
                    #print net.neighbors(v1)
                    break
            v1Neighbors = net.neighbors(v1)
            #print v1Neighbors
            #print v1
            #print v2
            #print(len(net.edges()))
            tobeDeleted = random.choice(v1Neighbors)
            net.remove_edge(v1, tobeDeleted)
            #print(len(net.edges()))
            #print([v1, tobeDeleted, v2])
            net.add_edge(v1, v2)
            numRewired = numRewired + 1
            #plotNet(net)
            #assert(0)
        return net, nx.to_numpy_matrix(net, dtype=np.float)

    elif type == 'smallworld':
        #net = nx.connected_watts_strogatz_graph(n, k, .15)
        net = nx.connected_watts_strogatz_graph(n, k, pRewire)
        return net, nx.to_numpy_matrix(net, dtype=np.float)  
    elif type == 'pref':
        net = nx.barabasi_albert_graph(n, 2)
        return net, nx.to_numpy_matrix(net, dtype=np.float)
    elif type == 'ER':
        #net = nx.erdos_renyi_graph(n, .006)
        targetDegree = 4.
        nEdgesPossible = ((n*n)-n)/2.
        pEdge = (n * targetDegree) / (2. * nEdgesPossible)
        assert(pEdge <= 1)

        # spare networks will likely be disconnected, so try a bunch
        tries = 100
        while tries > 0:
            net = nx.erdos_renyi_graph(n, pEdge)
            if nx.number_connected_components(net) > 1:
                tries = tries - 1
            else:
                break
        return net, nx.to_numpy_matrix(net, dtype=np.float)


In [6]:
def testThresh(agents, mn, mx):
    if np.mean(agents) > mx:
        return False
    if np.mean(agents) < mn:
        return False
    if np.mean(agents) <= 0:
        return False
    if np.mean(agents) >= 1:
        return False

    return True

In [7]:
def ltProp(agents, adjMatrix, nAgents, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = 0.
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)
    
    globalThreshold = .5
    if rs is None:
        thresholds = globalThreshold * np.random.random((1, nAgents))
    else:
        thresholds = globalThreshold * rs.random((1, nAgents))
    
    numNeighbors = np.sum(adjMatrix, axis=0)
    prevMean = -1
    step = 1
    
    while not testThresh(agents, haltMin, haltMax) and (np.mean(agents) > prevMean):
        #while np.mean(agents) > prevMean:
        prevMean = np.mean(agents)
        inp = np.true_divide(np.dot(agents, adjMatrix), numNeighbors)
        agents = np.logical_or(agents, (inp >= thresholds)).astype(int)

        step = step + 1
        
    #print('LT-proportional stopped at step ' + str(step) + ' '+ str(np.mean(agents)))
    return agents

In [8]:
def ltAbs(agents, adjMatrix, nAgents, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = 0.
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)

    # this controls the thresholds/pInfects for all contagion types
    globalThreshold = .5
    
    if rs is None:
        thresholds = np.random.randint(low=1, high=round(avgDegree*globalThreshold), size=(1, nAgents))
    else:
        thresholds = rs.randint(low=1, high=round(avgDegree*globalThreshold), size=(1, nAgents))
        
    numNeighbors = np.sum(adjMatrix, axis=0)
    #prevMean = -1
    step = 1
    
    while not testThresh(agents, haltMin, haltMax) and (np.mean(agents) > prevMean):
        prevMean = np.mean(agents)
        inp = np.dot(agents, adjMatrix)
        agents = np.logical_or(agents, (inp >= thresholds)).astype(int)

        step = step + 1
        
    # print('LT-absolute stopped at step ' + str(step) + ' '+ str(np.mean(agents)))

    return agents

In [9]:
def generateGraph(net, adjMatrix, pp, batch_size = 1, random_state=None):
    gg = []
    print(pp)
    while True:
        nAgents = nx.number_of_nodes(net)
        agents = np.zeros((1, nAgents))
        nodes = nx.nodes(net)
        seed_nodes = []
        inx = []
        count = 0
        for n in nodes:
            if n in seeds['ID'].tolist():
                seed_nodes.append(n)
                inx.append(count)
            count += 1
        
        agents[:,inx] = 1

        es_nodes = []
        es_inx = []
        count_es = 0
        num_es = 0
        for n in nodes:
            if n in es['ID'].tolist():
                es_nodes.append(n)
                es_inx.append(count_es)
                num_es += 1
            count_es += 1
    
        p_activation = num_es/count_es
        
        # seed neighbors of seeds
        for i in range(1):
            agents = np.logical_or(agents, np.dot(agents, adjMatrix)).astype(float) #all neighbors

        avgDegree = 2*net.number_of_edges() / float(net.number_of_nodes())

        if pp >= 0.5:
            agents_type = ltProp(agents, adjMatrix, nAgents, avgDegree=avgDegree, 
                                 haltMin=p_activation-0.025, haltMax=p_activation+0.025, rs = random_state)
#             agents_type = ltAbs(agents, adjMatrix, nAgents, avgDegree=avgDegree, 
#                                  haltMin=p_activation-0.025, haltMax=p_activation+0.025, rs = random_state)

        else:
            agents_type = ltAbs(agents, adjMatrix, nAgents, avgDegree=avgDegree, 
                                haltMin=p_activation-0.025, haltMax=p_activation+0.025, rs = random_state)

        #if not testThresh(agents_type, haltMin, haltMax):
         #   print('bad data, LTabs1:\t'+str(np.mean(agents_type)))
       #     continue
        gparam = {}
        gparam['init'] = net
        gparam['graph'] = agents_type
        
        gg.append(gparam['graph'])
        if len(gg) >= batch_size:
            break

    return gg[0]



In [10]:
def compileG(ll, batch_size = 1, random_state=None):
    return generateGraph(net_s, adjMat_s, ll[0], batch_size, random_state)


In [11]:
def yMetric(x, i = 0):
    return x[0,i]


In [12]:
def eucMultiArgs(X, Y):
    dist = np.linalg.norm(X - Y) 
    return np.array([dist])

In [26]:
seed = 20170530  # this will be separately given to ELFI
np.random.seed(seed)
N = 1000 # samples for rejection sampling
networktype = 'pref' #pref, smallworld, grid, ER, korea1, korea2, ckm
# nAgents = 4**2
# nAgents = 10**2
# sidelength = int(nAgents**.5) #10^2 = 100, 32^2 = 1024
#haltMin = .6 # minimum % of nodes active
#haltMax = .7 # maximum % of nodes active

# pRewire = .1 # proportion of edges to rewire
# numDatasets = 100 # number of data sets to generate
# saveData = True #save the output to file?


In [27]:
print(nx.__version__)
#parameters of the script
path = '../data/icpsr/DS0001/paluck-edgelist.csv'
edgelist = pd.read_csv(path)
G = nx.from_pandas_edgelist(edgelist, source='ID', target='PEERID')
print(nx.info(G))
print(nx.number_of_nodes(G))
print(f'connected?\t{nx.is_connected(G)}')
print(f'# of connected components:\t{nx.number_connected_components(G)}')

components = nx.connected_components(G)
sglist = [G.subgraph(c) for c in nx.connected_components(G)]

gmat = []
for g in sglist:
    gmat.append(nx.to_numpy_matrix(g, dtype=np.float))


2.5
Name: 
Type: Graph
Number of nodes: 23882
Number of edges: 132054
Average degree:  11.0589
23882
connected?	False
# of connected components:	57


In [ ]:
reslist = []

for ng in range(nx.number_connected_components(G)):
    #n,a = getGraphFromEdgelist(path)
    net_s = sglist[ng]
    adjMat_s = gmat[ng]
    nAgobs = nx.number_of_nodes(net_s)
    agobs = np.zeros((1, nAgobs))
    nodeobs = nx.nodes(net_s)
    es_nodeobs = []
    es_inxobs = []
    count_esobs = 0
    num_esobs = 0

    for n in nodeobs:
        if n in es['ID'].tolist():
            es_nodeobs.append(n)
            es_inxobs.append(count_esobs)
            num_esobs += 1
        count_esobs += 1

    agobs[:,es_inxobs] = 1

    gr_obs = np.matrix(agobs).astype(int)
    dcom = "d=elfi.Distance(eucMultiArgs"

    prop_prob = elfi.Prior(scipy.stats.bernoulli, 0.5)
    Y = elfi.Simulator(compileG, prop_prob, observed = gr_obs)
    ret = []
    for i in range(gr_obs.shape[1]):
        st = str(i)
        var_s = ''.join(['s',st])
        ret.append(var_s)
        com = var_s + ' = ' + 'elfi.Summary(yMetric, Y, '+st+')'
        exec(com)
        if i == 0:
            dcom = dcom + ',s' + str(i)
        else:
            dcom = dcom + ',s' + str(i)
    dcom = dcom + ')'

    exec(dcom)
    # d = elfi.Distance('euclidean',s7)
    # d = elfi.Distance('euclidean',s0, s1, s2, s3, s4, s5, s6)

    rej = elfi.Rejection(d, batch_size=1, seed=seed)
    %time result = rej.sample(N, quantile=0.1)

    reslist.append([ng+1, result.samples['prop_prob'].mean()])
    resdf = pd.DataFrame(reslist, columns = ["graph_index", "probability_parameter"])
    fname = "../results/rejection_sampling/N_" + str(N) + ".csv" 
    resdf.to_csv(fname, index = False)

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

...

0rogress [===-----------------------------------------------] 7.6% Complete
0rogress [===-----------------------------------------------] 7.6% Complete
1rogress [===-----------------------------------------------] 7.6% Complete
0rogress [===-----------------------------------------------] 7.6% Complete
0rogress [===-----------------------------------------------] 7.6% Complete
1rogress [===-----------------------------------------------] 7.6% Complete
0rogress [===-----------------------------------------------] 7.6% Complete
1rogress [===-----------------------------------------------] 7.6% Complete
1rogress [===-----------------------------------------------] 7.7% Complete
0rogress [===-----------------------------------------------] 7.7% Complete
0rogress [===-----------------------------------------------] 7.7% Complete
1rogress [===-----------------------------------------------] 7.7% Complete
1rogress [===-----------------------------------------------] 7.7% Complete
0rogress [==

0rogress [====----------------------------------------------] 8.7% Complete
1rogress [====----------------------------------------------] 8.7% Complete
0rogress [====----------------------------------------------] 8.7% Complete
1rogress [====----------------------------------------------] 8.7% Complete
0rogress [====----------------------------------------------] 8.7% Complete
1rogress [====----------------------------------------------] 8.7% Complete
1rogress [====----------------------------------------------] 8.7% Complete
0rogress [====----------------------------------------------] 8.7% Complete
0rogress [====----------------------------------------------] 8.7% Complete
0rogress [====----------------------------------------------] 8.8% Complete
0rogress [====----------------------------------------------] 8.8% Complete
1rogress [====----------------------------------------------] 8.8% Complete
0rogress [====----------------------------------------------] 8.8% Complete
1rogress [==

1rogress [====----------------------------------------------] 9.7% Complete
1rogress [====----------------------------------------------] 9.8% Complete
1rogress [====----------------------------------------------] 9.8% Complete
1rogress [====----------------------------------------------] 9.8% Complete
0rogress [====----------------------------------------------] 9.8% Complete
1rogress [====----------------------------------------------] 9.8% Complete
0rogress [====----------------------------------------------] 9.8% Complete
0rogress [====----------------------------------------------] 9.8% Complete
0rogress [====----------------------------------------------] 9.8% Complete
1rogress [====----------------------------------------------] 9.8% Complete
1rogress [====----------------------------------------------] 9.8% Complete
0rogress [====----------------------------------------------] 9.8% Complete
1rogress [====----------------------------------------------] 9.9% Complete
1rogress [==

0rogress [=====---------------------------------------------] 10.8% Complete
0rogress [=====---------------------------------------------] 10.8% Complete
0rogress [=====---------------------------------------------] 10.8% Complete
0rogress [=====---------------------------------------------] 10.8% Complete
1rogress [=====---------------------------------------------] 10.8% Complete
0rogress [=====---------------------------------------------] 10.9% Complete
1rogress [=====---------------------------------------------] 10.9% Complete
1rogress [=====---------------------------------------------] 10.9% Complete
0rogress [=====---------------------------------------------] 10.9% Complete
1rogress [=====---------------------------------------------] 10.9% Complete
0rogress [=====---------------------------------------------] 10.9% Complete
0rogress [=====---------------------------------------------] 10.9% Complete
1rogress [=====---------------------------------------------] 10.9% Complete

1rogress [=====---------------------------------------------] 11.9% Complete
0rogress [=====---------------------------------------------] 11.9% Complete
0rogress [=====---------------------------------------------] 11.9% Complete
1rogress [=====---------------------------------------------] 11.9% Complete
0rogress [=====---------------------------------------------] 11.9% Complete
0rogress [=====---------------------------------------------] 11.9% Complete
0rogress [=====---------------------------------------------] 11.9% Complete
0rogress [=====---------------------------------------------] 12.0% Complete
0rogress [=====---------------------------------------------] 12.0% Complete
1rogress [=====---------------------------------------------] 12.0% Complete
0rogress [=====---------------------------------------------] 12.0% Complete
0rogress [======--------------------------------------------] 12.0% Complete
1rogress [======--------------------------------------------] 12.0% Complete